In [4]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np

df = pd.read_csv("../Resources/Clean/aggregated_review_scores.csv")

In [2]:
bins = [0, 5.0, 7.9, df.abv.max()]
labels = ['Low ABV (0-5%)', 'Medium ABV (5-8%)', 'High ABV (8%+)']
df['abv_category'] = pd.cut(df.abv, bins=bins, labels=labels)

In [3]:
bins2 = [0, 20.0, 49.9, df.estimated_ibu.max()]
labels2 = ['Low Bitterness (0-20 IBU)', 'Medium Bitterness (20-50 IBU)', 'High Bitterness (50+ IBU)']
df['ibu_category'] = pd.cut(df.estimated_ibu, bins=bins2, labels=labels2)

In [4]:
df.head()

,beer_id,beer_name,beer_style,look,smell,taste,feel,overall,estimated_ibu,availability,abv,brewery_id,brewery_name,city,state,country,types,abv_category,ibu_category
0,203086,Hopper House Double IPA,American Imperial IPA,3.326923,3.346154,3.288462,3.307692,3.307692,83,Year-round,8.0,43368,Spindletap Brewery,Houston,TX,US,"Brewery, Bar",High ABV (8%+),High Bitterness (50+ IBU)
1,296880,Juiceton,New England IPA,4.330000,4.300000,4.330000,4.320000,4.310000,60,Rotating,8.5,43368,Spindletap Brewery,Houston,TX,US,"Brewery, Bar",High ABV (8%+),High Bitterness (50+ IBU)
2,319844,Rauchtabout,Irish Dry Stout,3.583333,3.500000,3.500000,3.416667,3.500000,35,Rotating,4.5,43368,Spindletap Brewery,Houston,TX,US,"Brewery, Bar",Low ABV (0-5%),Medium Bitterness (20-50 IBU)
3,356864,Hat Trick,New England IPA,4.200000,4.200000,4.100000,4.150000,4.200000,60,Rotating,7.0,43368,Spindletap Brewery,Houston,TX,US,"Brewery, Bar",Medium ABV (5-8%),High Bitterness (50+ IBU)
4,334690,Tiny Gypsies,New England IPA,4.500000,4.535714,4.571429,4.464286,4.571429,60,Rotating,6.5,43368,Spindletap Brewery,Houston,TX,US,"Brewery, Bar",Medium ABV (5-8%),High Bitterness (50+ IBU)


In [5]:
df_scaled = df.copy() 

In [6]:
df.look = df.look.round(2)
df.smell = df.smell.round(2)
df.taste = df.taste.round(2)
df.feel = df.feel.round(2)

In [7]:
# select features for the model
features = ['estimated_ibu', 'look', 'smell', 'taste', 'feel', 'abv']

# scale the features to a range between 0 and 1
scaler = MinMaxScaler()
df_scaled[features] = scaler.fit_transform(df_scaled[features])

# create a dataframe with only the relevant features
df_model = df_scaled[['beer_id', 'beer_name', 'beer_style', 'ibu_category'] + features + ['abv_category', 'brewery_name', 'city', 'state']]
df = df[['beer_name', 'beer_style', 'abv_category', 'ibu_category', 'availability', 'brewery_name', 'city', 'state', 'types']]

In [8]:
from sklearn.neighbors import NearestNeighbors

# define the number of nearest neighbors to consider
k = 10

# initialize the model with the number of neighbors
model = NearestNeighbors(n_neighbors=k)

# fit the model to the data
model.fit(df_model[features])

# define a function to recommend beers based on a given beer name
def recommend_beers_name(beer_name):
    # get the beer_id of the given beer name
    beer_id = df_model[df_model['beer_name'] == beer_name]['beer_id'].iloc[0]
    
    # get the index of the beer in the model dataframe
    idx = df_model[df_model['beer_id'] == beer_id].index[0]
    
    # get the features of the beer
    beer_features = df_model.loc[idx, features].values.reshape(1, -1)
    
    # find the k nearest neighbors
    distances, indices = model.kneighbors(beer_features)
    
    # get the beer names of the nearest neighbors
    beer_names = df.iloc[indices[0]]
    
    return beer_names

In [9]:
recommend_beers_name('700c Wit')

C:\Users\Gage\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,beer_name,beer_style,abv_category,ibu_category,availability,brewery_name,city,state,types
281,700c Wit,Belgian Witbier,Low ABV (0-5%),Low Bitterness (0-20 IBU),Rotating,Pinthouse Pizza Craft Brewpub,Austin,TX,"Brewery, Bar, Eatery, Beer-to-go"
1695,Cornerstone,American Cream Ale,Medium ABV (5-8%),Low Bitterness (0-20 IBU),Year-round,Solid Rock Brewing,Spicewood,TX,Brewery
2198,The Badge,American Blonde Ale,Low ABV (0-5%),Low Bitterness (0-20 IBU),Rotating,Nine-Band Brewing Company,Allen,TX,"Brewery, Bar"
1585,No Hitter,American Cream Ale,Low ABV (0-5%),Low Bitterness (0-20 IBU),Year-round,Idle Vine Brewing Co,Pflugerville,TX,"Brewery, Bar"
2179,Spring Break,Belgian Witbier,Medium ABV (5-8%),Low Bitterness (0-20 IBU),Spring,Audacity Brewhouse,Denton,TX,"Brewery, Bar, Eatery"
1893,Hung Jury Hefeweizen,German Hefeweizen,Low ABV (0-5%),Low Bitterness (0-20 IBU),Year-round,Legal Draft Beer Company,Arlington,TX,"Brewery, Bar, Beer-to-go"
1473,Berry White,Belgian Witbier,Low ABV (0-5%),Low Bitterness (0-20 IBU),Rotating,Whole Foods - Post Oak,Houston,TX,"Brewery, Bar, Eatery, Beer-to-go"
844,Tejas Clara,American Light Lager,Low ABV (0-5%),Low Bitterness (0-20 IBU),Rotating,Big Bend Brewing Co.,Alpine,TX,Brewery
10,Boomtown Blonde,American Blonde Ale,Medium ABV (5-8%),Low Bitterness (0-20 IBU),Year-round,Spindletap Brewery,Houston,TX,"Brewery, Bar"
1434,GRITz,American Cream Ale,Low ABV (0-5%),Low Bitterness (0-20 IBU),Year-round,Braindead Brewing,Dallas,TX,"Brewery, Bar, Eatery"


In [10]:
def recommend_beers_style(beer_style):
    # get the beer_ids of the given beer style
    beer_ids = df_model[df_model['beer_style'] == beer_style]['beer_id']
    
    # get the indices of the beers in the model dataframe
    indices = [df_model[df_model['beer_id'] == beer_id].index[0] for beer_id in beer_ids]
    
    # get the features of the beers
    beer_features = df_model.loc[indices, features]
    
    # find the k nearest neighbors
    distances, indices = model.kneighbors(beer_features)
    
    # get the beer names of the nearest neighbors
    beer_names = df.loc[indices[0]]
    
    return beer_names

In [11]:
recommend_beers_style("American IPA")

,beer_name,beer_style,abv_category,ibu_category,availability,brewery_name,city,state,types
6,Hop Option - El Dorado,American IPA,Medium ABV (5-8%),High Bitterness (50+ IBU),Rotating,Spindletap Brewery,Houston,TX,"Brewery, Bar"
1934,Alamo Holiday IPA,American IPA,Medium ABV (5-8%),High Bitterness (50+ IBU),Winter,Alamo Beer Co.,San Antonio,TX,"Brewery, Bar"
1075,Limestone,Belgian IPA,Medium ABV (5-8%),High Bitterness (50+ IBU),Rotating,Middleton Brewing,San Marcos,TX,"Brewery, Bar, Homebrew, Beer-to-go"
319,Pine Tree Boogie,American IPA,Medium ABV (5-8%),High Bitterness (50+ IBU),Rotating,Pinthouse Pizza Craft Brewpub,Austin,TX,"Brewery, Bar, Eatery, Beer-to-go"
2159,Destination,American IPA,Medium ABV (5-8%),High Bitterness (50+ IBU),Year-round,Vista Brewing,Driftwood,TX,"Brewery, Bar, Eatery, Beer-to-go"
121,Chupahopra,American IPA,Medium ABV (5-8%),High Bitterness (50+ IBU),Rotating,Twisted X Brewing Company,Dripping Springs,TX,"Brewery, Bar"
187,Mór IPA,American IPA,Medium ABV (5-8%),High Bitterness (50+ IBU),Year-round,Shannon Brewing Company,Keller,TX,"Brewery, Bar, Beer-to-go"
1688,Fresh Coast,American IPA,Medium ABV (5-8%),High Bitterness (50+ IBU),Rotating,Friends & Allies Brewing,Austin,TX,"Brewery, Bar"
342,IPA Ver. 2 (Reformulated),American IPA,Medium ABV (5-8%),High Bitterness (50+ IBU),Year-round,Tupps Brewery,McKinney,TX,"Brewery, Bar"
1051,SMaSH Belma,American IPA,Medium ABV (5-8%),High Bitterness (50+ IBU),Rotating,Last Stand Brewing Company,Austin,TX,"Brewery, Bar"


In [12]:
def recommend_beers_abv(abv_category, ibu_category):
    # get the beer_ids of the given beer style
    beer_ids = df_model[(df_model['abv_category'] == abv_category) & 
                        (df_model['ibu_category'] == ibu_category)]['beer_id']
    
    # get the indices of the beers in the model dataframe
    indices = [df_model[df_model['beer_id'] == beer_id].index[0] for beer_id in beer_ids]
    
    # get the features of the beers
    beer_features = df_model.loc[indices, features]
    
    # find the k nearest neighbors
    distances, indices = model.kneighbors(beer_features)
    
    # get the beer names of the nearest neighbors
    beer_names = df.loc[indices[0]]
    
    return beer_names

In [13]:
recommend_beers_abv("High ABV (8%+)", "Low Bitterness (0-20 IBU)")

,beer_name,beer_style,abv_category,ibu_category,availability,brewery_name,city,state,types
24,Pecan Pie,Scottish Ale,High ABV (8%+),Low Bitterness (0-20 IBU),Rotating,Buffalo Bayou Brewing Co,Houston,TX,"Brewery, Bar"
1900,Act Of Balor,English Sweet / Milk Stout,High ABV (8%+),Low Bitterness (0-20 IBU),Winter,Back Pew Brewing Company,Porter,TX,Brewery
800,The Regulator,German Doppelbock,High ABV (8%+),Medium Bitterness (20-50 IBU),Spring,Rahr & Sons Brewing Company,Fort Worth,TX,"Brewery, Bar"
1633,(512) Wild Bear,American Wild Ale,High ABV (8%+),Low Bitterness (0-20 IBU),Rotating,(512) Brewing Company,Austin,TX,"Brewery, Bar"
272,Cosmic Dancer,American Wild Ale,High ABV (8%+),Low Bitterness (0-20 IBU),Rotating,New Braunfels Brewing Company,New Braunfels,TX,"Brewery, Bar"
299,The Dude,English Sweet / Milk Stout,High ABV (8%+),Low Bitterness (0-20 IBU),Rotating,Pinthouse Pizza Craft Brewpub,Austin,TX,"Brewery, Bar, Eatery, Beer-to-go"
137,Vintage Monks 2013,Belgian Dubbel,High ABV (8%+),Medium Bitterness (20-50 IBU),Rotating,Adelbert's Brewery,Austin,TX,"Brewery, Bar"
1203,Mysterium Verum Rex Indomitus,American Wild Ale,High ABV (8%+),Low Bitterness (0-20 IBU),Rotating,Real Ale Brewing Company,Blanco,TX,"Brewery, Bar"
1100,Feisty Blonde,American Blonde Ale,High ABV (8%+),Low Bitterness (0-20 IBU),Year-round,HopFusion Ale Works,Fort Worth,TX,Brewery
417,Triple Dunkel,German Dunkelweizen,High ABV (8%+),Low Bitterness (0-20 IBU),Year-round,Franconia Brewing Company,McKinney,TX,Brewery
